# Noteboook for testing view classification

In [ ]:
import sys
import os
import pandas as pd
import datetime

projectdir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(projectdir, 'src'))

from d00_utils.db_utils import dbReadWriteClassification
from d02_intermediate.download_dcm import s3_download_decomp_dcm
from d02_intermediate.dcm_utils import dcmdir_to_jpgs_for_classification
import d03_classification.predict_view as predict_view

In [ ]:
dcm_dir_suffix = s3_download_decomp_dcm(0.001, 0.5, 'instances_w_labels')

In [ ]:
dcm_dir = os.path.join(os.path.expanduser('~/data/01_raw/'), dcm_dir_suffix)
img_dir = os.path.join(os.path.expanduser('~/data/02_intermediate/'), dcm_dir_suffix)
#dcmdir_to_jpgs_for_classification(dcm_dir, img_dir)

In [ ]:
model_path = '/home/ubuntu/models/view_23_e5_class_11-Mar-2018'
feature_dim = 1
if_exists='replace'
predictions = predict_view.run_classify(img_dir, model_path, if_exists, feature_dim)

In [ ]:
io_classification = dbReadWriteClassification()
io_classification.list_tables()

In [ ]:
df = io_classification.get_table('predictions')
df.head()

In [ ]:
predict_view.agg_predictions()

In [ ]:
#to overwrite existing predictions table
#io_classification.save_to_db(df.iloc[:0,:-1], "predictions", if_exists="replace")

In [ ]:
io_classification.raw_conn.rollback() #to reset after error